### Imports

In [30]:
from pyod.models.ecod import ECOD
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.51 ms (started: 2023-07-11 15:09:14 -05:00)


### Parameters

In [31]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

exp = params["exp"]
print("Exp:\t\t", exp)

method = params["method"]
print("Method:\t\t", method)

dimension = params["dimension"]
print("Dimension:\t", dimension)

groups_id = params["groups_id"]
print("Groups id:\t", groups_id)

subgroups_id = params["subgroups_id"]
print("Subgroups id:\t", subgroups_id)

option = params["option"]
print("Option:\t\t", option)

if option:
    
    subgroups_id_op = {}
    for group in groups_id:
        subgroups_id_op[group] = [option]
else:
    subgroups_id_op = subgroups_id
print("Subgroups id op:", subgroups_id_op)



Exp:		 exp8
Method:		 vgae
Dimension:	 3
Groups id:	 ['Starved', 'FCSglc', 'DMA']
Subgroups id:	 {'Starved': ['1', '2'], 'FCSglc': ['1', '2'], 'DMA': ['1', '2']}
Option:		 dyn
Subgroups id op: {'Starved': ['dyn'], 'FCSglc': ['dyn'], 'DMA': ['dyn']}
time: 13.4 ms (started: 2023-07-11 15:09:14 -05:00)


### Edge embeddings

In [32]:
# get edges embeddings

edge_embeddings_global(exp, method, groups_id, subgroups_id_op)

100%|██████████| 3/3 [14:51<00:00, 297.22s/it]

time: 14min 51s (started: 2023-07-11 15:09:14 -05:00)


In [33]:
df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, groups_id[0], subgroups_id_op[groups_id[0]][0]), index_col=[0, 1])
df_edge_embeddings.head()

0         1         2
0 1  0.225743  0.111096  4.325736
  2  0.194221  0.010269  0.588489
  3  0.497761  0.111119  0.256925
  4  1.995459  0.338965  0.365798
  5  1.986327  0.150453  6.438817

time: 2.13 s (started: 2023-07-11 15:24:05 -05:00)


### Concat edge embeddings

In [34]:
for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.DataFrame()
    k = 0
    for subgroup in tqdm(subgroups_id_op[group]):
        df_edge_embeddings = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_{}_{}_{}.csv".format(exp, method, group, subgroup), index_col=[0, 1])
        df_edge_embeddings["subgroup"] = [k] * len(df_edge_embeddings)
        df_edge_embeddings_concat = pd.concat([df_edge_embeddings_concat, df_edge_embeddings])
        k += 1
    
    df_edge_embeddings_concat.to_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index=True)

100%|██████████| 3/3 [01:58<00:00, 39.64s/it]

time: 1min 58s (started: 2023-07-11 15:24:07 -05:00)


In [35]:
df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, groups_id[0], option), index_col=[0, 1])
df_edge_embeddings_concat.head()

0         1         2  subgroup
0 1  0.225743  0.111096  4.325736         0
  2  0.194221  0.010269  0.588489         0
  3  0.497761  0.111119  0.256925         0
  4  1.995459  0.338965  0.365798         0
  5  1.986327  0.150453  6.438817         0

time: 2.17 s (started: 2023-07-11 15:26:06 -05:00)


In [36]:
# plot edge embeddings concat

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    x = df_edge_embeddings_concat.iloc[:, 0]
    y = df_edge_embeddings_concat.iloc[:, 1]
    z = df_edge_embeddings_concat.iloc[:, 2]

    # Creating figure
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection="3d")

    # Creating plot
    ax.scatter3D(x, y, z, c=df_edge_embeddings_concat.iloc[:, -1], alpha=0.1)
    # plt.title("Dimension: {}".format(dimension))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_concat_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

100%|██████████| 3/3 [07:07<00:00, 142.40s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 7min 7s (started: 2023-07-11 15:26:09 -05:00)


### Outliers detection

In [37]:
# Outlier detection (HDBSCAN)

""" df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])

X_train = df_edge_embeddings_concat.iloc[:, :-1]
clusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)

threshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)
outliers = np.where(clusterer.outlier_scores_ > threshold)[0]
print(len(outliers))
outliers

inliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)
print(len(inliers))
inliers """

' df_edge_embeddings_concat = pd.read_csv("output/edge_embeddings/edge-embeddings_concat_{}_{}_{}_{}.csv".format(group, method, dimension, "L2"), index_col=[0, 1])\n\nX_train = df_edge_embeddings_concat.iloc[:, :-1]\nclusterer = hdbscan.HDBSCAN(min_cluster_size=100, core_dist_n_jobs=-1).fit(X_train)\n\nthreshold = pd.Series(clusterer.outlier_scores_).quantile(0.9)\noutliers = np.where(clusterer.outlier_scores_ > threshold)[0]\nprint(len(outliers))\noutliers\n\ninliers = np.setdiff1d(np.arange(len(df_edge_embeddings_concat)), outliers)\nprint(len(inliers))\ninliers '

time: 2.68 ms (started: 2023-07-11 15:33:16 -05:00)


In [38]:
# outlier detection (ECOD)

dict_df_edge_embeddings_concat_outlier = {}
dict_df_edge_embeddings_concat_filter = {}

for group in tqdm(groups_id):
    df_edge_embeddings_concat = pd.read_csv("output/{}/edge_embeddings/edge-embeddings_concat_{}_{}_{}.csv".format(exp, method, group, option), index_col=[0, 1])

    X_train = df_edge_embeddings_concat.iloc[:, :-1]

    clf = ECOD()
    clf.fit(X_train)

    X_train["labels"] = clf.labels_ # binary labels (0: inliers, 1: outliers)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat.copy()
    df_edge_embeddings_concat_filter["labels"] = clf.labels_
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["labels"] == 0]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, :-1]

    dict_df_edge_embeddings_concat_outlier[group] = X_train
    dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:26<00:00,  8.73s/it]

time: 26.3 s (started: 2023-07-11 15:33:16 -05:00)


In [39]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
0 1  0.225743  0.111096  4.325736         0
  2  0.194221  0.010269  0.588489         0
  3  0.497761  0.111119  0.256925         0
  4  1.995459  0.338965  0.365798         0
  6  0.382072  1.410285  0.956025         0

time: 6.02 ms (started: 2023-07-11 15:33:42 -05:00)


In [40]:
# plot outliers/inliers

for group in tqdm(groups_id):
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")

    df_aux = dict_df_edge_embeddings_concat_outlier[group]
    print("Total:", len(df_aux))
    
    temp = df_aux[df_aux["labels"] == 0]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="red", alpha=0.005)
    print("Num. of inliers:", len(temp))

    temp = df_aux[df_aux["labels"] == 1]
    x = temp.iloc[:, 0]
    y = temp.iloc[:, 1]
    z = temp.iloc[:, 2]
    ax.scatter3D(x, y, z, c="gray", alpha=0.005)
    print("Num. of inliers:", len(temp))

    # show plot
    plt.savefig("output/{}/plots/edge-embeddings_outlier_{}_{}_{}.png".format(exp, method, group, option))
    # plt.show()
    plt.clf()

  0%|          | 0/3 [00:00<?, ?it/s]

Total: 6054117


Num. of inliers: 5448705
Num. of inliers: 605412


 33%|███▎      | 1/3 [02:20<04:40, 140.45s/it]

Total: 5898362
Num. of inliers: 5308525
Num. of inliers: 589837


...

100%|██████████| 3/3 [06:55<00:00, 138.36s/it]


<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

<Figure size 1000x700 with 0 Axes>

time: 6min 55s (started: 2023-07-11 15:33:42 -05:00)


###  Filter common edges

In [41]:
print(subgroups_id)
print(subgroups_id_op)

{'Starved': ['1', '2'], 'FCSglc': ['1', '2'], 'DMA': ['1', '2']}
{'Starved': ['dyn'], 'FCSglc': ['dyn'], 'DMA': ['dyn']}
time: 718 µs (started: 2023-07-11 15:40:37 -05:00)


In [42]:
# mapping idx with id

for group in tqdm(groups_id):
    dict_df_nodes = {}
    for subgroup in subgroups_id_op[group]:
        df_nodes = pd.read_csv("output/{}/preprocessing/graphs_data/nodes_data_{}_{}.csv".format(exp, group, subgroup),
                               dtype={"id": "string"})
        dict_df_nodes[subgroup] = df_nodes
    
    # mapping
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
    list_index = []
    
    for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
        df_nodes = dict_df_nodes[subgroups_id_op[group][row[-1]]]
        list_index.append((df_nodes.iloc[row[0][0], -1], df_nodes.iloc[row[0][1], -1]))
    
    # set new index
    df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)

  0%|          | 0/3 [00:00<?, ?it/s]

5448705it [02:23, 38099.33it/s]
5308525it [02:19, 38137.80it/s]58, 149.43s/it]
5488576it [02:25, 37682.30it/s]26, 146.52s/it]
100%|██████████| 3/3 [07:24<00:00, 148.23s/it]

time: 7min 24s (started: 2023-07-11 15:40:37 -05:00)


In [43]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
A59.01425 A61.98612  0.225743  0.111096  4.325736         0
          A71.01184  0.194221  0.010269  0.588489         0
          A74.00508  0.497761  0.111119  0.256925         0
          A74.02387  1.995459  0.338965  0.365798         0
          A82.02924  0.382072  1.410285  0.956025         0

time: 11.4 ms (started: 2023-07-11 15:48:02 -05:00)


In [44]:
# format id
if option:
    for group in tqdm(groups_id):
        # format
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        list_index = []

        for row in tqdm(df_edge_embeddings_concat_filter.itertuples()):
            list_index.append((row[0][0][1:], row[0][1][1:]))
        
        # set new index
        df_edge_embeddings_concat_filter.set_index([pd.Index(list_index)], inplace=True)
        df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

5448705it [00:05, 1011650.52it/s]
5308525it [00:04, 1115197.92it/s], 10.91s/it]
5488576it [00:04, 1166003.23it/s], 11.36s/it]
100%|██████████| 3/3 [00:33<00:00, 11.09s/it]

time: 33.3 s (started: 2023-07-11 15:48:02 -05:00)


In [45]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
df_edge_embeddings_concat_filter.head()

0         1         2  subgroup
59.01425 61.98612  0.225743  0.111096  4.325736         0
         71.01184  0.194221  0.010269  0.588489         0
         74.00508  0.497761  0.111119  0.256925         0
         74.02387  1.995459  0.338965  0.365798         0
         82.02924  0.382072  1.410285  0.956025         0

time: 5.84 ms (started: 2023-07-11 15:48:35 -05:00)


In [46]:
# filter diferente edges

if option:
    for group in tqdm(groups_id):
        df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]
        df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter.index.get_level_values(0) != df_edge_embeddings_concat_filter.index.get_level_values(1)]
        dict_df_edge_embeddings_concat_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  1.84it/s]

time: 1.65 s (started: 2023-07-11 15:48:35 -05:00)


In [47]:
df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[groups_id[0]]
print(df_edge_embeddings_concat_filter.shape)
df_edge_embeddings_concat_filter.head()

(5446733, 4)


0         1         2  subgroup
59.01425 61.98612  0.225743  0.111096  4.325736         0
         71.01184  0.194221  0.010269  0.588489         0
         74.00508  0.497761  0.111119  0.256925         0
         74.02387  1.995459  0.338965  0.365798         0
         82.02924  0.382072  1.410285  0.956025         0

time: 5.63 ms (started: 2023-07-11 15:48:37 -05:00)


In [48]:
# count edges and filter by count

dict_df_edges_filter = {}
for group in tqdm(groups_id):
    # read
    df_edge_embeddings_concat_filter = dict_df_edge_embeddings_concat_filter[group]

    # format
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.rename(columns={"level_0": "source", "level_1": "target"}, inplace=True)
    df_edge_embeddings_concat_filter[["source", "target"]] = df_edge_embeddings_concat_filter[["source", "target"]].astype("string")
    
    # sort edges
    sort_df_edges(df_edge_embeddings_concat_filter)

    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[["source", "target"]].value_counts().to_frame()
    df_edge_embeddings_concat_filter.reset_index(inplace=True)
    df_edge_embeddings_concat_filter.columns = ["source", "target", "count"]
    
    # filter
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter[df_edge_embeddings_concat_filter["count"] == len(subgroups_id[group])]
    df_edge_embeddings_concat_filter = df_edge_embeddings_concat_filter.iloc[:, [0, 1]]
    dict_df_edges_filter[group] = df_edge_embeddings_concat_filter

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:25<00:00,  8.48s/it]

time: 25.5 s (started: 2023-07-11 15:48:37 -05:00)


In [49]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,286.99155,319.00522
1,339.07562,615.25543
2,339.07562,613.13422
3,339.07562,613.14508
4,339.07562,614.05371


time: 4.21 ms (started: 2023-07-11 15:49:02 -05:00)


In [50]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2406688 entries, 0 to 2406687
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   source  object
 1   target  object
dtypes: object(2)
memory usage: 55.1+ MB
time: 19.7 ms (started: 2023-07-11 15:49:02 -05:00)


In [51]:
# change data type
for group in tqdm(groups_id):
    df_edges_filter = dict_df_edges_filter[group]
    df_edges_filter[["source", "target"]] = df_edges_filter[["source", "target"]].astype("string")

 33%|███▎      | 1/3 [00:00<00:00,  4.95it/s]

100%|██████████| 3/3 [00:00<00:00,  5.42it/s]

time: 568 ms (started: 2023-07-11 15:49:02 -05:00)


In [52]:
df_edges_filter = dict_df_edges_filter[groups_id[0]]
df_edges_filter.head()

,source,target
0,286.99155,319.00522
1,339.07562,615.25543
2,339.07562,613.13422
3,339.07562,613.14508
4,339.07562,614.05371


time: 4.11 ms (started: 2023-07-11 15:49:03 -05:00)


In [53]:
df_edges_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2406688 entries, 0 to 2406687
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   source  string
 1   target  string
dtypes: string(2)
memory usage: 55.1 MB
time: 17.3 ms (started: 2023-07-11 15:49:03 -05:00)


In [54]:
# get weight by subgroups

dict_df_edges_filter_weight = get_weight_global(dict_df_edges_filter, exp, groups_id, subgroups_id)
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:45<00:00, 15.28s/it]


,source,target,subgroup1,subgroup2
1785792,100.00072,1004.6228,-1.0,1.0
1785793,100.00072,1015.62897,-1.0,-1.0
1785794,100.00072,102.98343,-1.0,1.0
1785795,100.00072,1033.12769,-1.0,1.0
1785796,100.00072,1033.14172,-1.0,1.0


time: 46 s (started: 2023-07-11 15:49:03 -05:00)


In [55]:
df_edges_filter_weight = dict_df_edges_filter_weight[groups_id[0]]
df_edges_filter_weight.head()

,source,target,subgroup1,subgroup2
1785792,100.00072,1004.6228,-1.0,1.0
1785793,100.00072,1015.62897,-1.0,-1.0
1785794,100.00072,102.98343,-1.0,1.0
1785795,100.00072,1033.12769,-1.0,1.0
1785796,100.00072,1033.14172,-1.0,1.0


time: 4.81 ms (started: 2023-07-11 15:49:49 -05:00)


### Filter by STD and average weight

In [56]:
dict_df_common_edges = std_global(dict_df_edges_filter_weight, exp, method, groups_id, option, th=0.3, plot=True, save=True)
dict_df_common_edges[groups_id[0]].head()

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:48<00:00, 16.10s/it]


,source,target,weight
0,100.00072,1015.62897,-1.0
1,100.00072,1033.15576,1.0
2,100.00072,1039.01721,1.0
3,100.00072,1040.8479,1.0
4,100.00072,1045.24829,1.0


<Figure size 640x480 with 0 Axes>

time: 48.3 s (started: 2023-07-11 15:49:49 -05:00)


In [57]:
df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, groups_id[0], option),
                              dtype={"source": "string", "target": "string"})
df_common_edges.head()

,source,target,weight
0,100.00072,1015.62897,-1.0
1,100.00072,1033.15576,1.0
2,100.00072,1039.01721,1.0
3,100.00072,1040.8479,1.0
4,100.00072,1045.24829,1.0


time: 343 ms (started: 2023-07-11 15:50:37 -05:00)


In [58]:
# show details

for group in tqdm(groups_id):
    df_common_edges = pd.read_csv("output/{}/common_edges/common_edges_{}_{}_{}.csv".format(exp, method, group, option))
    
    G = nx.from_pandas_edgelist(df_common_edges, "source", "target", edge_attr=["weight"])
    print("Group: {}".format(group))
    graph_detail(G)

 33%|███▎      | 1/3 [00:02<00:04,  2.27s/it]

Group: Starved
Num. nodes: 2427
Num. edges: 1204011



 67%|██████▋   | 2/3 [00:04<00:02,  2.24s/it]

Group: FCSglc
Num. nodes: 2365
Num. edges: 1148220



100%|██████████| 3/3 [00:06<00:00,  2.30s/it]

Group: DMA
Num. nodes: 2445
Num. edges: 1259008

time: 6.91 s (started: 2023-07-11 15:50:38 -05:00)
